# Value Investing Automation

## 1. Load Libraries

In [1]:
import csv
import requests
import numpy as np
from time import sleep

## 2. Set some initial parameters

In [2]:
# Minimum market capitalization, to weed out small-cap stocks
min_cap = 300000000
# API key required to access teh financial modeling prep services
api_key = 'api key here'

## 3. Grab a list of NYSE stock tickers

Grabbed from a generic .csv file containing almost all stocks listed on the NYSE

In [3]:
stocks_raw = []

In [53]:
ticker_to_name = {}

with open("nyse-listed.csv", mode='r') as csv_file:
    # open the .csv and iterate through the rows
    csv_reader = csv.DictReader(csv_file)
    line = 0
    for row in csv_reader:
        line += 1
        if '$' in row['ACT Symbol']:
            # '$' character is found in some weird listings in the .csv file.
            # Just ignore these listings
            continue
        if line == 0:
            # first (zeroeth) line is just column labels, ignore them
            continue
        
        # put the ticker into a list if it matches the above criteria.
        # Called an 'ACT Symbol' for some reason in this .csv
        stocks_raw.append(row['ACT Symbol'])
        ticker_to_name[row['ACT Symbol']] = row['Company Name']

## 4. Define functions to grab the market capitalization (CAP), return on invested capital (ROIC), and price to earnings ratio (PE)

In [5]:
def getCAP(stock, api_key):
    try:
        # Try to grab the information directly
        evalues = requests.get(f"https://financialmodelingprep.com/api/v3/enterprise-values/{stock}?limit=1&apikey={api_key}").json()
    except:
        # If attempt fails, it's most likely we've exceeded the number of API calls allowed
        # within a specific amount of time. Wait a few minutes and then try again.
        print("getCAP: too many requests! Pausing for 5 minutes")
        sleep(300)
        evalues = requests.get(f"https://financialmodelingprep.com/api/v3/enterprise-values/{stock}?limit=1&apikey={api_key}").json()
    try:
        # try to return the CAP. It is sometimes possible that the CAP is not available
        # for a given ticker. If that is the case, return Nothing
        return evalues[0]['marketCapitalization']
    except:
        return None

# ============================= Same logic applies on lower 2 functions ======================================== #
    
def getROIC(stock, api_key):
    try:
        key_metrics = requests.get(f"https://financialmodelingprep.com/api/v3/key-metrics-ttm/{stock}?limit=1&apikey={api_key}").json()
    except:
        print("getROIC: too many requests! Pausing for 5 minutes")
        sleep(300)
        key_metrics = requests.get(f"https://financialmodelingprep.com/api/v3/key-metrics-ttm/{stock}?limit=1&apikey={api_key}").json()
    try:
        return key_metrics[0]['roicTTM']
    except:
        return None
    

def getPE(stock, api_key):
    try:
        ratios = requests.get(f"https://financialmodelingprep.com/api/v3/ratios-ttm/{stock}?limit=1&apikey={api_key}").json()
    except:
        print("getPE: too many requests! Pausing for 5 minutes")
        sleep(300)
        ratios = requests.get(f"https://financialmodelingprep.com/api/v3/ratios-ttm/{stock}?limit=1&apikey={api_key}").json()
    try:
        return ratios[0]['peRatioTTM']
    except:
        return None

# Perform some tests to ensure that the above functions work,
# grab some basic information on Apple Inc.
print("(Example) Apple market cap:\t" + str(getCAP('AAPL', api_key)))
print("(Example) Apple ROIC:\t\t" + str(getROIC('AAPL', api_key)))
print("(Example) Apple PE:\t\t" + str(getPE('AAPL', api_key)))

(Example) Apple market cap:	1996361343006.3572
(Example) Apple ROIC:		0.33740921790861533
(Example) Apple PE:		31.782818357891443


## 5. Remove the tickers with market caps below the pre-specified threshold

In [6]:
stocks = []
for s in stocks_raw:
    # Grab the market cap for a ticker
    cap = getCAP(s, api_key)
    
    # If the market capitalization cannot be found, don't add it. Just skip it.
    if not cap:
        continue
        
    # If the ticker's market cap is above the threshold, add it.
    if cap >= min_cap:
        stocks.append(s)

## 6. Grab the PE & ROIC for remaining tickers

In [7]:
stockdat = []
for s in stocks:
    sdat = {}
    roic = getROIC(s, api_key)    # Grab the ROIC
    pe = getPE(s, api_key)        # Grab the PE ratio
    
    # If either the ROIC or PE ratio cannot be found for a
    # ticker, ignore it.
    if (roic is None) or (pe is None):
        continue
        
    # Add the pe, roic, and name to a dictionary object and
    # append it to a list of stock data
    sdat['pe'] = pe
    sdat['roic'] = roic
    sdat['name'] = s
    
    stockdat.append(sdat)

getPE: too many requests! Pausing for 5 minutes


## 7. Sort the tickers by general rank

Set some initial parameters to further sort results

In [22]:
pe_min = 0.0             # minimum price to earnings ratio
pe_max = 60.0            # maximum price to earnings ratio

roic_min = 0.06          # minimum ROIC
roic_max = 9999999999    # maximum ROIC

In [58]:
# Generate an argsort list to organize the PE ratios from lowest to highest
pe_list_argsort = np.argsort(np.array([x['pe'] for x in stockdat]))

# Generate an argsort list to organize the ROIC from highest to lowest
roic_list_argsort = np.flip(np.argsort(np.array([x['roic'] for x in stockdat])))

# Attach the PE ratio & ROIC rankings to each ticker.
# Also calculate a "general" rank by adding each PE & ROIC
# ranking.
for i in range(len(stockdat)):
    stockdat[i]['pe_rank'] = pe_list_argsort[i]
    stockdat[i]['roic_rank'] = roic_list_argsort[i]
    stockdat[i]['rank'] = pe_list_argsort[i] + roic_list_argsort[i]
    
    stockdat[i]['Full Name'] = ticker_to_name[stockdat[i]['name']]
    stockdat[i]['Website'] =  f"https://finance.yahoo.com/quote/{sdat['name']}?p={sdat['name']}&.tsrc=fin-srch"

# Generate an argsort list to organize stock tickers by "general rank"
rank_argsort = np.argsort(np.array([x['rank'] for x in stockdat]))


# Print all stock tickers in order of "general" rank, assuming
# they're within the thresholds
print("Name \t|\t rank \t|\t PE \t|\t ROIC \t|\t Website")
print("=====================================================================================================")
for i in range(len(stockdat)):
    sdat = stockdat[rank_argsort[i]]
    if (sdat['pe'] <= pe_min) or (sdat['pe'] >= pe_max):
        continue
    if (sdat['roic'] <= roic_min) or (sdat['roic'] >= roic_max):
        continue
    print(str(sdat['name']) +
          "\t|\t" +
          str(sdat['rank']) +
          "\t|\t" +
          "{0:.3f}".format(sdat['pe']) +
          "x\t|\t" +
          "{0:.2%}".format(sdat['roic']) +
          "\t|\t" +
          ticker_to_name[sdat['name']] + " " +
          f"https://finance.yahoo.com/quote/{sdat['name']}?p={sdat['name']}&.tsrc=fin-srch")

Name 	|	 rank 	|	 PE 	|	 ROIC 	|	 Website
ELS	|	100	|	48.550x	|	6.05%	|	Equity Lifestyle Properties, Inc. Common Stock https://finance.yahoo.com/quote/ELS?p=ELS&.tsrc=fin-srch
MO	|	111	|	19.945x	|	17.97%	|	Altria Group, Inc. https://finance.yahoo.com/quote/MO?p=MO&.tsrc=fin-srch
BAX	|	116	|	35.935x	|	7.74%	|	Baxter International Inc. Common Stock https://finance.yahoo.com/quote/BAX?p=BAX&.tsrc=fin-srch
GGG	|	131	|	30.405x	|	22.48%	|	Graco Inc. Common Stock https://finance.yahoo.com/quote/GGG?p=GGG&.tsrc=fin-srch
BTT	|	135	|	8.244x	|	8.48%	|	BlackRock Municipal Target Term Trust Inc. (The) Common Shares of Beneficial Interest https://finance.yahoo.com/quote/BTT?p=BTT&.tsrc=fin-srch
NAC	|	144	|	5.945x	|	9.73%	|	Nuveen California Dividend Advantage Municipal Fund Common Stock https://finance.yahoo.com/quote/NAC?p=NAC&.tsrc=fin-srch
DG	|	161	|	18.558x	|	16.12%	|	Dollar General Corporation Common Stock https://finance.yahoo.com/quote/DG?p=DG&.tsrc=fin-srch
SYF	|	163	|	11.699x	|	9.79%	|	Sync

## Generate & export a .csv file for further analysis

In [60]:
with open('value_stock_analysis.csv', mode='w', newline='') as csv_file:
    fieldnames = ['name', 'pe', 'roic', 'pe_rank', 'roic_rank', 'rank', 'Full Name', 'Website']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    writer.writeheader()
    for row in stockdat:
        writer.writerow(row)